In [1]:
import pandas as pd
import numpy as np
from faker import Faker
from random import choice, choices, randrange
from joblib import Parallel, delayed
from unidecode import unidecode
import datetime
import ast
from datetime import datetime, timedelta
import os

In [2]:
from src import BOOK_GenBehaviour as bkbeh
from src import BOOK_Grouping as bkgrp
from src import BOOK_SOIGrouping as bksoi

In [3]:
poi_base = pd.read_csv('data/POI_base.csv')
cw = pd.read_csv('data/geoCrosswalk/GeoCrossWalkMed.csv')
df_flight = pd.read_csv('data/flightData/EU_flight_new.csv')
agencies = pd.read_csv('data/agencies.csv')['agencies'].tolist()
agency_weight = pd.read_csv('data/agencies.csv')['agency_weight'].tolist()
route = pd.read_csv('data/flightData/route_all.csv')


In [4]:
poi_base

,SOI_Type,Mark,AgeMin_Main,AgeMax_Main,MF_Prob,HH_ISO_List,SizeHH,SizeHH_Prob,IATA_O_List,IATA_D_List,Lang_List,stay_day,stay_day_weight
0,HTP,Perpetrator,25,50,"[0.7, 0.3]","['USA', 'CAN']",2,1,"['CDG', 'LHR']","['AUH', 'DXB']","['en_US', 'no_NO', 'ar_SA']","[1,2,3]","[0.7, 0.2, 0.1]"
1,HTV,Victim,5,15,"[0.6, 0.4]","['GBR', 'DZA']",1,1,"['IST', 'FRA']","['LHR', 'SVO']","['en_US', 'no_NO', 'ar_SA']","[1,2,3]","[0.7, 0.2, 0.1]"
2,DSM,Perpetrator,19,55,"[0.7, 0.3]","['IND', 'DEU']","[1, 2]","[0.7, 0.3]","['CDG', 'LHR']","['MXP', 'DXB']","['en_IN', 'de_DE']","[1,2,3]","[0.7, 0.2, 0.1]"
3,TRR,Perpetrator,20,25,"[0.6, 0.4]","['IDN', 'ITA']","[1, 2]","[0.6, 0.4]","['STN', 'MUC']","['AMS', 'SVO']","['id_ID', 'it_IT']","[1,2,3]","[0.7, 0.2, 0.1]"


In [5]:
def list_in_column(df, column):
    df[column] = df[column].str.replace("[", "")
    df[column] = df[column].str.replace("]", "")
    df[column] = df[column].str.replace("'", "")
    df[column] = df[column].str.replace(" ", "")
    df[column] = df[column].str.split(',')
    return df.copy()

In [6]:
poi_base_clean = poi_base.copy()
for column in ['HH_ISO_List', 'SizeHH', 'SizeHH_Prob', 'IATA_O_List', 'IATA_D_List', 'Lang_List', 'stay_day', 'stay_day_weight']:
    poi_base_clearn = list_in_column(poi_base_clean, column)

In [7]:
df = poi_base_clean.copy()

In [8]:
        
def safe_locale_gen(locale):
    try:
        return Faker(locale)
    except AttributeError:
        return Faker('en')

def generate_hh(hh_num, df):
    hh_list = []
    for i in range(hh_num):
        hh = {}
        hh['HH_num'] = i
        SOI_Type_List = df['SOI_Type'].dropna().unique().tolist()
        if not SOI_Type_List:
            continue
        SOI = choice(SOI_Type_List)
        row = df[df['SOI_Type'] == SOI]
        if row.empty:
            continue
        
        # Checking and converting the MF_Prob values
        if not row['MF_Prob'].isna().any():
            sex = ['M', 'F']
            # Check if the data is string and needs parsing
            probs = [float(x) for x in (ast.literal_eval(row['MF_Prob'].values[0]) if isinstance(row['MF_Prob'].values[0], str) else row['MF_Prob'].values[0])]
            hh['GenderHOH'] = choices(sex, probs)[0] if len(sex) == len(probs) else ''

        # Ensure AgeHOH values are integers and not NaN
        if not row['AgeMin_Main'].isna().any() and not row['AgeMax_Main'].isna().any():
            hh['AgeHOH'] = randrange(int(row['AgeMin_Main'].values[0]), int(row['AgeMax_Main'].values[0]) + 1)

        # Checking and converting the SizeHH and SizeHH_Prob values
        if not row['SizeHH'].isna().any() and not row['SizeHH_Prob'].isna().any():
            sizes = [int(x) for x in row['SizeHH'].values[0]]
            probs = [float(x) for x in (ast.literal_eval(row['SizeHH_Prob'].values[0]) if isinstance(row['SizeHH_Prob'].values[0], str) else row['SizeHH_Prob'].values[0])]
            print(sizes, probs)
            hh['SizeHH'] = choices(sizes, probs)[0] if len(sizes) == len(probs) else ''

        # Simplify remaining attribute assignments
        hh['HHID'] = SOI + '_' + str(i)
        hh['HH_ISO'] = choice(row['HH_ISO_List'].dropna().values[0]) if not row['HH_ISO_List'].isna().all() else ''
        hh['HHType'] = SOI
        hh['Lang'] = choice(row['Lang_List'].dropna().values[0]) if not row['Lang_List'].isna().all() else ''
        faker_gen = safe_locale_gen(hh['Lang'])
        
        
        hh.update({
            'Address': faker_gen.address(),
            'PostCode': faker_gen.postcode(),
            'Country': faker_gen.current_country(),
            'PaymentInfo_VendorCode': faker_gen.credit_card_provider(),
            'PaymentInfo_ExpiryDate': faker_gen.credit_card_expire(start="now", end="+10y", date_format="%d/%m/%y"),
            'PaymentInfo_AccountNbr': faker_gen.credit_card_number(card_type=None)
        })
        hh['ISO_Travel'] = ''
        hh['IATA_O'] = choice(row['IATA_O_List'].dropna().values[0]) if not row['IATA_O_List'].isna().all() else ''
        hh_list.append(hh)
    
    return pd.DataFrame(hh_list)

        
        



In [9]:
df

,SOI_Type,Mark,AgeMin_Main,AgeMax_Main,MF_Prob,HH_ISO_List,SizeHH,SizeHH_Prob,IATA_O_List,IATA_D_List,Lang_List,stay_day,stay_day_weight
0,HTP,Perpetrator,25,50,"[0.7, 0.3]","[USA, CAN]",[2],[1],"[CDG, LHR]","[AUH, DXB]","[en_US, no_NO, ar_SA]","[1, 2, 3]","[0.7, 0.2, 0.1]"
1,HTV,Victim,5,15,"[0.6, 0.4]","[GBR, DZA]",[1],[1],"[IST, FRA]","[LHR, SVO]","[en_US, no_NO, ar_SA]","[1, 2, 3]","[0.7, 0.2, 0.1]"
2,DSM,Perpetrator,19,55,"[0.7, 0.3]","[IND, DEU]","[1, 2]","[0.7, 0.3]","[CDG, LHR]","[MXP, DXB]","[en_IN, de_DE]","[1, 2, 3]","[0.7, 0.2, 0.1]"
3,TRR,Perpetrator,20,25,"[0.6, 0.4]","[IDN, ITA]","[1, 2]","[0.6, 0.4]","[STN, MUC]","[AMS, SVO]","[id_ID, it_IT]","[1, 2, 3]","[0.7, 0.2, 0.1]"


In [10]:
df_HH = generate_hh(10, df)
df_HH

[1, 2] [0.7, 0.3]
[1] [1.0]
[1] [1.0]
[1] [1.0]
[1, 2] [0.7, 0.3]
[1] [1.0]
[1, 2] [0.6, 0.4]
[1, 2] [0.7, 0.3]
[1] [1.0]
[1, 2] [0.7, 0.3]


,HH_num,GenderHOH,AgeHOH,SizeHH,HHID,HH_ISO,HHType,Lang,Address,PostCode,Country,PaymentInfo_VendorCode,PaymentInfo_ExpiryDate,PaymentInfo_AccountNbr,ISO_Travel,IATA_O
0,0,M,33,2,DSM_0,DEU,DSM,en_IN,"H.No. 550, Banik Road, Munger 672944",880820,India,VISA 16 digit,11/08/27,4579579701520368237,,CDG
1,1,F,9,1,HTV_1,DZA,HTV,en_US,"PSC 4534, Box 5339\nAPO AA 28701",13120,United States,Maestro,09/02/33,3547358672941131,,IST
2,2,M,9,1,HTV_2,DZA,HTV,ar_SA,"94910 الدباغ Fort Suite 298\nفاديton, WV 12378",40773,United States,Discover,08/01/34,502090244109,,FRA
3,3,M,15,1,HTV_3,GBR,HTV,ar_SA,"876 جواهر Meadows\nمهناstad, AK 22598",80839,United States,JCB 15 digit,06/05/25,180000449965583,,FRA
4,4,F,40,1,DSM_4,DEU,DSM,de_DE,Andersgasse 190\n64052 Fallingbostel,85742,Germany,VISA 19 digit,02/12/33,2248055680807791,,LHR
5,5,M,6,1,HTV_5,DZA,HTV,en_US,"273 Burke Causeway\nEast Jonathanfurt, CO 33518",33046,United States,Diners Club / Carte Blanche,01/06/33,2269096913610523,,FRA
6,6,M,24,1,TRR_6,ITA,TRR,id_ID,"Gg. Pasirkoja No. 4\nParepare, Kalimantan Timu...",14616,Indonesia,Diners Club / Carte Blanche,30/07/31,372794283112348,,STN
7,7,F,47,1,DSM_7,DEU,DSM,en_IN,H.No. 13\nMalhotra\nTumkur-113253,638021,India,VISA 19 digit,23/03/31,4086766766801372,,LHR
8,8,M,12,1,HTV_8,DZA,HTV,ar_SA,"78616 ضاحي Wall Apt. 901\nالمهيدبville, VI 52016",87326,United States,Discover,27/01/26,3559718222701324,,IST
9,9,M,24,1,DSM_9,DEU,DSM,de_DE,Stumpfallee 1/6\n06655 Main-Höchst,74521,Germany,American Express,25/02/30,6011618000261126,,LHR


In [11]:
def populate_persons(household_row, df_city, poi_base):
    """
    Populate passengers for a given household, ensuring diversity and generating comprehensive passenger attributes.
    Now includes a 20% chance for typology names to differ from document names.
    
    Parameters:
    - household_row: Series, a row from the household DataFrame.
    - df_city: DataFrame, contains city data for determining locales.
    
    Returns:
    - List of comprehensive passenger data for the household.
    """

    passengers = []
    HHID = household_row['HHID']
    HHType = household_row['HHType']
    base_age = household_row['AgeHOH']
    lang = household_row['Lang']
    payment_vendor = household_row['PaymentInfo_VendorCode']
    payment_expiry = household_row['PaymentInfo_ExpiryDate']
    payment_number = household_row['PaymentInfo_AccountNbr']
    ISO_Travel = household_row['ISO_Travel']
    IATA_O = household_row['IATA_O']    
    faker_gen = safe_locale_gen(lang)
    row_HTV = poi_base[poi_base['SOI_Type'] == 'HTV']
    row_group = poi_base[poi_base['SOI_Type'] == HHType]

    for j in range(household_row['SizeHH']):
        P_num = j
        P_ID = f"{HHID}_{j+1}"
        if HHType == 'HTP' or j == 0:
            age = base_age
            gender = household_row['GenderHOH']
        elif HHType == 'HTP' and j > 0:
            age = randrange(int(row_HTV['AgeMin_Main'].values[0]), int(row_HTV['AgeMax_Main'].values[0]) + 1)
            sex_list = ['M', 'F']
            probs = [float(x) for x in (ast.literal_eval(row_HTV['MF_Prob'].values[0]) if isinstance(row_HTV['MF_Prob'].values[0], str) else row_group['MF_Prob'].values[0])]
            gender =  choices(sex_list, probs)[0] if len(sex_list) == len(probs) else ''
        else:
            age = randrange(int(row_group['AgeMin_Main'].values[0]), int(row_group['AgeMax_Main'].values[0]) + 1)
            sex_list = ['M', 'F']
            probs = [float(x) for x in (ast.literal_eval(row_group['MF_Prob'].values[0]) if isinstance(row_group['MF_Prob'].values[0], str) else row_group['MF_Prob'].values[0])]
            gender =  choices(sex_list, probs)[0] if len(sex_list) == len(probs) else ''

        first_name = faker_gen.first_name_male() if gender == 'M' else faker_gen.first_name_female()
        surname = faker_gen.last_name()
        dob = generate_dob(age)
        free_email = faker_gen.free_email()
        # payment_vendor = faker_gen.credit_card_provider()
        # payment_expiry = faker_gen.credit_card_expire(start="now", end="+10y", date_format="%d/%m/%y")
        # payment_number = faker_gen.credit_card_number(card_type=None)
        work_email = faker_gen.company_email()
        docs_expiry = (datetime.today() + timedelta(days=365 * 10)).strftime('%Y-%m-%d')  # Assuming 10 years from now
        
        doc_first_name = unidecode(first_name)
        doc_surname = unidecode(surname)  # Document names are the real names
        typ_first_name, typ_surname = generate_typ_names(faker_gen, doc_first_name, doc_surname)  # Generate TYP names
        NationalityNat = choice(df_city['HH_ISO'].tolist())
        



        passenger = [
            P_num, HHID, P_ID, age, 
            f"{5 * (age // 5)}-{5 * (age // 5) + 4}" if age < 100 else "100+",  # AgeRange
            f"AGE{age // 5 + 1}" if age < 100 else "AGE21",  # AgeGroup
            gender, household_row['GenderHOH'], base_age, household_row['SizeHH'],
            household_row['HH_ISO'], HHType, lang, surname, household_row['Address'],
            household_row['PostCode'], household_row['Country'], first_name, dob, free_email,
            payment_vendor, payment_expiry, payment_number, work_email, docs_expiry,
            doc_first_name, doc_surname, typ_first_name, typ_surname, NationalityNat, ISO_Travel, IATA_O
            ]
    # No placeholder for P_num is added here
        passengers.append(passenger)
    
    return passengers

def generate_person_data(df_HH, df_city, df):
    """
    Generate comprehensive passenger data for all households.
    
    Parameters:
    - df_HH: DataFrame, household data.
    - df_city: DataFrame, city data for locale information.
    
    Returns:
    - DataFrame with comprehensive passenger data.
    """
    # passenger_data = Parallel(n_jobs=-1)(delayed(populate_passengers)(row, df_city) for index, row in df_HH.iterrows())
    # passenger_data = [p for sublist in passenger_data for p in sublist]  # Flatten the list of lists

    passenger_data = []
    for index, row in df_HH.iterrows():
        passengers = populate_persons(row, df_city, df)
        passenger_data.extend(passengers) 
    
    columns = [
        'P_num', 'HHID', 'P_ID', 'P_AGE', 'AgeRange', 'AgeGroup', 'P_GENDER', 'GenderHOH', 'AgeHOH', 'SizeHH', 'HH_ISO', 'HHType', 'Lang',
        'Surname', 'Address', 'PostCode', 'Country', 'FirstName', 'DOB', 'FreeEmail',
        'PaymentInfo_VendorCode', 'PaymentInfo_ExpiryDate', 'PaymentInfo_AccountNbr',
        'WorkEmail', 'DOCS_ExpiryDate', 'DOC_FirstName', 'DOC_Surname', 'TYP_FirstName',
        'TYP_Surname', 'NationalityNat', 'ISO_Travel', 'IATA_O'
    ]
    df_passengers = pd.DataFrame(passenger_data, columns=columns)
    
    return df_passengers

def finalize_data(df):
    """
    Finalize the DataFrame by adding P_num and ensuring the correct column order.
    """
    df.insert(0, 'P_num', range(1, len(df) + 1))  # Insert P_num at the beginning
    column_order = [
        'P_num', 'HHID', 'P_ID', 'P_AGE', 'AgeRange', 'AgeGroup', 'P_GENDER', 'GenderHOH', 'AgeHOH', 'SizeHH', 'HH_ISO', 'HHType', 'Lang',
        'Surname', 'Address', 'PostCode', 'Country', 'FirstName', 'DOB', 'FreeEmail', 'PaymentInfo_VendorCode', 'PaymentInfo_ExpiryDate',
        'PaymentInfo_AccountNbr', 'WorkEmail', 'DOCS_ExpiryDate', 'DOC_FirstName', 'DOC_Surname', 'TYP_FirstName', 'TYP_Surname'
    ]
    return df[column_order]

In [12]:
def introduce_typos(text, typo_rate):
    typo_text = list(text)
    for i in range(len(typo_text)-1):
        if np.random() < typo_rate:
            # Introduce a typo (e.g., swap with the next character)
            typo_text[i], typo_text[i+1] = typo_text[i+1], typo_text[i]
    return ''.join(typo_text)

def docIDs(row):

    data={}
    random_number = np.random()
    if random_number > 0.005:
        data['TYP_FirstName'] = row['DOC_FirstName']
        data['TYP_Surname'] = row['DOC_Surname']
        # data['TYP_DOB'] = str(row['DOB'])
    else:
        data['TYP_FirstName'] = introduce_typos(row['DOC_FirstName'], typo_rate=0.2)
        data['TYP_Surname'] = introduce_typos(row['DOC_Surname'], typo_rate=0.2)
        # data['TYP_DOB'] = introduce_dob_typos2(str(row['DOB']), typo_rate=0.2)
    
    return data

def generate_dob(age):
    """
    Generate a Date of Birth for the given age.
    """
    today = datetime.today()
    start_of_year = datetime(today.year - age, 1, 1)
    end_of_year = datetime(today.year - age, 12, 31)
    random_days = timedelta(days=(end_of_year - start_of_year).days * np.random.random())
    dob = start_of_year + random_days
    return dob.strftime('%d/%m/%Y')


def generate_typ_names(faker_gen, doc_first_name, doc_surname):
    """
    Generate typology names with a 20% chance of being different from the document names.
    """
    if np.random.rand() < 0.5:  # 20% chance
        typ_first_name = faker_gen.first_name()
        typ_surname = faker_gen.last_name()
    else:
        typ_first_name = doc_first_name
        typ_surname = doc_surname
    return typ_first_name, typ_surname

In [13]:
df_passenger = generate_person_data(df_HH, cw, df)

In [14]:
df_passenger

,P_num,HHID,P_ID,P_AGE,AgeRange,AgeGroup,P_GENDER,GenderHOH,AgeHOH,SizeHH,...,PaymentInfo_AccountNbr,WorkEmail,DOCS_ExpiryDate,DOC_FirstName,DOC_Surname,TYP_FirstName,TYP_Surname,NationalityNat,ISO_Travel,IATA_O
0,0,DSM_0,DSM_0_1,33,30-34,AGE7,M,M,33,2,...,4579579701520368237,bariafaiyaz@sane-das.net,2034-05-01,Hansh,Doshi,Shalv,Choudhary,CHN,,CDG
1,1,DSM_0,DSM_0_2,47,45-49,AGE10,M,M,33,2,...,4579579701520368237,ibarman@choudhury-thakkar.com,2034-05-01,Krish,Mani,Krish,Mani,USA,,CDG
2,0,HTV_1,HTV_1_1,9,5-9,AGE2,F,F,9,1,...,3547358672941131,christopher42@myers-ball.info,2034-05-01,Robin,Rivera,Robin,Rivera,DEU,,IST
3,0,HTV_2,HTV_2_1,9,5-9,AGE2,M,M,9,1,...,502090244109,al-ydyqwb@al.info,2034-05-01,mrshd,Hnbwly,mrshd,Hnbwly,FRA,,FRA
4,0,HTV_3,HTV_3_1,15,15-19,AGE4,M,M,15,1,...,180000449965583,smy57@bqshn.com,2034-05-01,wy'l,al `yD,مريم,شربتلي,IRQ,,FRA
5,0,DSM_4,DSM_4_1,40,40-44,AGE9,F,F,40,1,...,2248055680807791,lorchemmi@loewer.de,2034-05-01,Erdmute,Kabus,Erdmute,Kabus,KOR,,LHR
6,0,HTV_5,HTV_5_1,6,5-9,AGE2,M,M,6,1,...,2269096913610523,thanson@wilson-king.biz,2034-05-01,Gregory,Adams,Billy,Donovan,FRA,,FRA
7,0,TRR_6,TRR_6_1,24,20-24,AGE5,M,M,24,1,...,372794283112348,usadaluhung@pd.gov,2034-05-01,Hendri,Mayasari,Hendri,Mayasari,USA,,STN
8,0,DSM_7,DSM_7_1,47,45-49,AGE10,F,F,47,1,...,4086766766801372,lallzeeshan@venkatesh-madan.com,2034-05-01,Azad,Yadav,Mehul,Kota,UKR,,LHR
9,0,HTV_8,HTV_8_1,12,10-14,AGE3,M,M,12,1,...,3559718222701324,rshydlmgwl@al.info,2034-05-01,khDr,'bw dwwd,ناظم,أبو داوود,EGY,,IST


In [15]:
df_behaviour_complete = bkbeh.generate_behaviour(df_HH, df_flight, ['SOI'], [1], 3, cw, 1)

Destination Assigned
Traveller Type Assigned
Destination Assigned
Traveller Type Assigned
Destination Assigned
Traveller Type Assigned
Behaviour Complete


In [16]:
df_behaviour_complete.head()

,HH_num,GenderHOH,AgeHOH,SizeHH,HHID,HH_ISO,HHType,Lang,Address,PostCode,Country,PaymentInfo_VendorCode,PaymentInfo_ExpiryDate,PaymentInfo_AccountNbr,ISO_Travel,IATA_O,Behaviour_1,Behaviour_2,Behaviour_3
0,0,M,33,2,DSM_0,DEU,DSM,en_IN,"H.No. 550, Banik Road, Munger 672944",880820,India,VISA 16 digit,11/08/27,4579579701520368237,,CDG,"[ORY, SOI, SOI, 2]","[LHR, SOI, SOI, 2]","[KEF, SOI, SOI, 2]"
1,1,F,9,1,HTV_1,DZA,HTV,en_US,"PSC 4534, Box 5339\nAPO AA 28701",13120,United States,Maestro,09/02/33,3547358672941131,,IST,"[ORK, SOI, SOI, 1]","[SVO, SOI, SOI, 1]","[CPH, SOI, SOI, 1]"
2,2,M,9,1,HTV_2,DZA,HTV,ar_SA,"94910 الدباغ Fort Suite 298\nفاديton, WV 12378",40773,United States,Discover,08/01/34,502090244109,,FRA,"[MAD, SOI, SOI, 1]","[DUB, SOI, SOI, 1]","[LHR, SOI, SOI, 1]"
3,3,M,15,1,HTV_3,GBR,HTV,ar_SA,"876 جواهر Meadows\nمهناstad, AK 22598",80839,United States,JCB 15 digit,06/05/25,180000449965583,,FRA,"[TLS, SOI, SOI, 1]","[GVA, SOI, SOI, 1]","[LHR, SOI, SOI, 1]"
4,4,F,40,1,DSM_4,DEU,DSM,de_DE,Andersgasse 190\n64052 Fallingbostel,85742,Germany,VISA 19 digit,02/12/33,2248055680807791,,LHR,"[LGW, SOI, SOI, 1]","[DUS, SOI, SOI, 1]","[CDG, SOI, SOI, 1]"


In [17]:
select_behaviour = 'Behaviour_1'
df_group = bksoi.grouping_init(df_behaviour_complete, select_behaviour, agencies, agency_weight, route, poi_base_clean, 1)

done SOI
[1, 2, 3] [0.7, 0.2, 0.1]
[1, 2, 3] [0.7, 0.2, 0.1]
[1, 2, 3] [0.7, 0.2, 0.1]
[1, 2, 3] [0.7, 0.2, 0.1]


In [18]:
df_group

,init_id,list_of_passengers,route,num_in_party,BookingAgency,BookingDay,Unnamed: 0,IATA_O,IATA_D,HH_ISO_D,...,region_O,LegHH_ISO,LegRegion,Origin,Destination,fixRoute,Leg_fixRoute,stay_day,airport_itinerary_out,full_routes
0,FRA-MAD-SOI-ID0,[HTV_2_1],FRA-MAD,1,Airlines,12,564.0,FRA,MAD,ESP,...,Europe,DEU-ESP,Europe-Europe,FRA,MAD,FRA->MAD,1,1,"[FRA, MAD]",[FRA-MAD]
1,FRA-TLS-SOI-ID0,[HTV_3_1],FRA-TLS,1,BookingAgency2,9,638.0,FRA,TLS,FRA,...,Europe,DEU-FRA,Europe-Europe,FRA,TLS,FRA->TLS,1,1,"[FRA, TLS]",[FRA-TLS]
2,FRA-LHR-SOI-ID0,[HTV_5_1],FRA-LHR,1,Airlines,26,553.0,FRA,LHR,GBR,...,Europe,DEU-GBR,Europe-Europe,FRA,LHR,FRA->LHR,1,1,"[FRA, LHR]",[FRA-LHR]
3,LHR-BUD-SOI-ID0,[DSM_7_1],LHR-BUD,1,Airlines,27,1271.0,LHR,BUD,HUN,...,Europe,GBR-HUN,Europe-Europe,LHR,BUD,LHR->BUD,1,3,"[LHR, BUD]",[LHR-BUD]
